In [3]:
root_dir = "../../Downloads/GENOME540/"
protein = "bacterioferritins" # "transthyretins" "ferritins" "hemoglobins" "insulins"
fastas_path = root_dir + protein + ".txt"
score_matrix_path = root_dir + "blosum62.txt"
output_path = root_dir + "_wdag.txt"

In [4]:
def read_fastas(filename):
    fastas = {}
    with open(filename, 'r') as f:
        lines = f.read().split('>')
        lines = [line for line in lines if line]
        for line in lines:
            title, seq = line.split('\n', 1)
            fastas[title] = seq.replace('\n', '')
    fastas = list(fastas.values())
    return fastas, list(set("".join(fastas))) + ["-"]

fastas, aminos = read_fastas(fastas_path)
fastas

['MKGDTKVINYLNKLLGNELVAINQYFLHARMFKNWGLKRLNDVEYHESIDEMKHADRYIERILFLEGLPNLQDLGKLNIGEDVEEMLRSDLALELDGAKNLREAIGYADSVHDYVSRDMMIEILRDEEGHIDWLETELDLIQKMGLQNYLQAQIREEG',
 'MKGDAKVIEFLNAALRSELTAISQYWVHFRLQEDWGLAKMAKKSREESIEEMGHADKIIARILFLEGHPNLQKLDPLRIGEGPRETLECDLAGEHDALKLYREARDYCAEVGDIVSKNIFESLITDEEGHVDFLETQISLYDRLGPQGFALLNAAPMDAAE',
 'MAGNREDRKAKVIEVLNKARAMELHAIHQYMNQHYSLDDMDYGELAANMKLIAIDEMRHAENFAERIKELGGEPTTQKEGKVVTGQAVPVIYESDADQEDATIEAYSQFLKVCKEQGDIVTARLFERIIEEEQAHLTYYENIGSHIKNLGDTYLAKIAGTPSSTGTASKGFVTATPAAE']

In [5]:
def read_score_matrix(filename):
    with open(filename, 'r') as f:
        amino_acids = f.readline().strip().split()
        score_matrix = {}
        for line in f:
            scores = line.strip().split()
            amino_acid = scores[0]
            score_matrix[amino_acid] = {}
            for i in range(len(amino_acids)):
                score_matrix[amino_acid][amino_acids[i]] = int(scores[i + 1])
    return score_matrix

blosum62 = read_score_matrix(score_matrix_path)
gap_pen = -6

In [6]:
def get_weight(edge, score_matrix, gap_penalty):
    x1, x2, x3 = edge
    pair_weights = [0, 0, 0]
    if x1 != "-" and x2 != "-":
        pair_weights[0] = score_matrix[x1][x2]
    elif x1 != "-":
        pair_weights[0] = gap_penalty
    elif x2 != "-":
        pair_weights[0] = gap_penalty
    if x1 != "-" and x3 != "-":
        pair_weights[1] = score_matrix[x1][x3]
    elif x1 != "-":
        pair_weights[1] = gap_penalty
    elif x3 != "-":
        pair_weights[1] = gap_penalty
    if x2 != "-" and x3 != "-":
        pair_weights[2] = score_matrix[x2][x3]
    elif x2 != "-":
        pair_weights[2] = gap_penalty
    elif x3 != "-":
        pair_weights[2] = gap_penalty
    return sum(pair_weights)

In [7]:
import itertools

def display_weights(letters, score_matrix, gap_penalty):
    edges = sorted(["".join(x) for x in list(itertools.product(letters, repeat=3))])[1:]
    print("Edge weights:")
    for edge in edges:
        print(edge, get_weight(edge, score_matrix, gap_penalty))

display_weights(aminos, blosum62, gap_pen)

Edge weights:
--A -12
--C -12
--D -12
--E -12
--F -12
--G -12
--H -12
--I -12
--K -12
--L -12
--M -12
--N -12
--P -12
--Q -12
--R -12
--S -12
--T -12
--V -12
--W -12
--Y -12
-A- -12
-AA -8
-AC -12
-AD -14
-AE -13
-AF -14
-AG -12
-AH -14
-AI -13
-AK -13
-AL -13
-AM -13
-AN -14
-AP -13
-AQ -13
-AR -13
-AS -11
-AT -12
-AV -12
-AW -15
-AY -14
-C- -12
-CA -12
-CC -3
-CD -15
-CE -16
-CF -14
-CG -15
-CH -15
-CI -13
-CK -15
-CL -13
-CM -13
-CN -15
-CP -15
-CQ -15
-CR -15
-CS -13
-CT -13
-CV -13
-CW -14
-CY -14
-D- -12
-DA -14
-DC -15
-DD -6
-DE -10
-DF -15
-DG -13
-DH -13
-DI -15
-DK -13
-DL -16
-DM -15
-DN -11
-DP -13
-DQ -12
-DR -14
-DS -12
-DT -13
-DV -15
-DW -16
-DY -15
-E- -12
-EA -13
-EC -16
-ED -10
-EE -7
-EF -15
-EG -14
-EH -12
-EI -15
-EK -11
-EL -15
-EM -14
-EN -12
-EP -13
-EQ -10
-ER -12
-ES -12
-ET -13
-EV -14
-EW -15
-EY -14
-F- -12
-FA -14
-FC -14
-FD -15
-FE -15
-FF -6
-FG -15
-FH -13
-FI -12
-FK -15
-FL -12
-FM -12
-FN -15
-FP -16
-FQ -15
-FR -15
-FS -14
-FT -14
-FV -13
-FW -11

In [8]:
class Vertex:
    def __init__(self, label):
        self.label = label
        self.edges = []

    def add_edge(self, edge):
        self.edges.append(edge)

    def sort_edges(self):
        self.edges = sorted(self.edges, key=lambda edge: -edge.weight)

class Edge:
    def __init__(self, label, start, end, weight):
        self.label = label
        self.start = start
        self.end = end
        self.weight = weight

    def __lt__(self, other):
        return self.weight < other.weight

In [9]:
def create_graph(sequences, score_matrix, gap_penalty):
    seq1, seq2, seq3 = sequences[0], sequences[1], sequences[2]
    n1, n2, n3 = len(seq1), len(seq2), len(seq3)

    vertices = [[[Vertex(f"{i}_{j}_{k}") for k in range(n3 + 1)] for j in range(n2 + 1)] for i in range(n1 + 1)]

    for i in range(n1 + 1):
        for j in range(n2 + 1):
            for k in range(n3 + 1):
                vertex = vertices[i][j][k]

                if i < n1:
                    edge_label = seq1[i] + "-" + "-"
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i + 1][j][k]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if j < n2:
                    edge_label = "-" + seq2[j] + "-"
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i][j + 1][k]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if k < n3:
                    edge_label = "-" + "-" + seq3[k]
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i][j][k + 1]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if i < n1 and j < n2:
                    edge_label = seq1[i] + seq2[j] + "-"
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i + 1][j + 1][k]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if j < n2 and k < n3:
                    edge_label = "-" + seq2[j] + seq3[k]
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i][j + 1][k + 1]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if i < n1 and k < n3:
                    edge_label = seq1[i] + "-" + seq3[k]
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i + 1][j][k + 1]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if i < n1 and j < n2 and k < n3:
                    edge_label = seq1[i] + seq2[j] + seq3[k]
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i + 1][j + 1][k + 1]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

    return [vertex for row in vertices for col in row for vertex in col]

verts = create_graph(fastas, blosum62, gap_pen)

In [10]:
def write_graph(vertices, filename, write=False):
    if write:
        with open(filename, "w") as f:
            for vertex in vertices:
                f.write(f"V {vertex.label}\n")
                for edge in vertex.edges:
                    f.write(f"E {edge.start.label} {edge.end.label} {edge.weight}\n")

write_graph(verts, output_path)

In [11]:
from collections import Counter

print("Edge counts:")
counts = sorted(Counter([y.label for x in verts for y in x.edges]).items())
for count in counts:
    print(count[0], count[1])

Edge counts:
--A 566676
--C 25758
--D 257580
--E 515160
--F 103032
--G 309096
--H 154548
--I 334854
--K 309096
--L 283338
--M 154548
--N 180306
--P 103032
--Q 206064
--R 180306
--S 180306
--T 309096
--V 231822
--Y 206064
-A- 486540
-AA 59466
-AC 2703
-AD 27030
-AE 54060
-AF 10812
-AG 32436
-AH 16218
-AI 35139
-AK 32436
-AL 29733
-AM 16218
-AN 18921
-AP 10812
-AQ 21624
-AR 18921
-AS 18921
-AT 32436
-AV 24327
-AY 21624
-C- 57240
-CA 6996
-CC 318
-CD 3180
-CE 6360
-CF 1272
-CG 3816
-CH 1908
-CI 4134
-CK 3816
-CL 3498
-CM 1908
-CN 2226
-CP 1272
-CQ 2544
-CR 2226
-CS 2226
-CT 3816
-CV 2862
-CY 2544
-D- 343440
-DA 41976
-DC 1908
-DD 19080
-DE 38160
-DF 7632
-DG 22896
-DH 11448
-DI 24804
-DK 22896
-DL 20988
-DM 11448
-DN 13356
-DP 7632
-DQ 15264
-DR 13356
-DS 13356
-DT 22896
-DV 17172
-DY 15264
-E- 543780
-EA 66462
-EC 3021
-ED 30210
-EE 60420
-EF 12084
-EG 36252
-EH 18126
-EI 39273
-EK 36252
-EL 33231
-EM 18126
-EN 21147
-EP 12084
-EQ 24168
-ER 21147
-ES 21147
-ET 36252
-EV 27189
-EY 24168
-

In [12]:
from collections import deque

def topological_sort(vertices):
    indegree = {v: 0 for v in vertices}
    for v in vertices:
        for e in v.edges:
            indegree[e.end] += 1
    
    queue = deque(v for v in vertices if indegree[v] == 0)
    
    result = []
    while queue:
        v = queue.popleft()
        result.append(v)
        for e in v.edges:
            indegree[e.end] -= 1
            if indegree[e.end] == 0:
                queue.append(e.end)
    
    if len(result) != len(vertices):
        raise RuntimeError("bad graph")
    
    return result

def get_path(vertex, longest_paths):
    path = []
    while longest_paths[vertex] != 0:
        max_edge = max(vertex.edges, key=lambda e: longest_paths[e.end] + e.weight)
        path.append(max_edge)
        vertex = max_edge.end
    return path[::-1]

def highest_weight_path(vertices):

    sorted_vertices = topological_sort(vertices)
    
    longest_paths = {v: 0 for v in vertices}
    
    for v in reversed(sorted_vertices):
        longest_path = 0
        for e in v.edges:
            longest_path = max(longest_path, longest_paths[e.end] + e.weight)
        longest_paths[v] = longest_path
    
    highest_weight = max(longest_paths.values())
    highest_path = []
    for v in vertices:
        if longest_paths[v] == highest_weight:
            highest_path = get_path(v, longest_paths)
            break
    
    if highest_path:
        return highest_weight, [x.label for x in highest_path][::-1]
    else:
        return 0, []

score, path = highest_weight_path(verts)

print("Score:", score)
print("Local alignment:")
print("\n".join(path))

Score: 676
Local alignment:
MME
KKD
GGR
DDK
TAA
KKK
VVV
III
NEE
YFV
LLL
NNN
KAK
LAA
LLR
GRA
NSM
EEE
LLL
VTH
AAA
III
NSH
QQQ
YYY
FWM
--N
LVQ
HHH
AFY
RRS
MLL
FQ-
KED
NDD
WWM
GGD
LLY
KAG
RKE
LML
NAA
DKA
VKN
ESM
YRK
HEL
EEI
SSA
III
DED
EEE
MMM
KGR
HHH
AAA
DDE
RKN
YIF
IIA
EAE
RRR
III
LLK
FFE
LLL
EEG
GGG
LHE
PPP
NNT
LLT
QQQ
DKK
LLE
GDG
KPK
LLV
NRV
IIT
GGG
EEQ
DGA
VPV
ERP
EEV
MTI
LLY
REE
SCS
DDD
LL-
AAA
LGD
EEQ
LHE
DDD
GAA
--T
ALI
KKE
NLA
LYY
RRS
EEQ
AAF
IRL
GDK
YYV
ACC
DAK
SEE
VVQ
HGG
DDD
YII
VVV
SST
RKA
DNR
MIL
MFF
IEE
ESR
ILI
LII
RTE
DDE
EEE
EEQ
GGA
HHH
IVL
DDT
WFY
LLY
EEE
TTN
EQI
LIG
DSS
LLH
IYI
QDK
KRN
MLL
GGG
LP-
QQD
NGT
YFY
LAL
